In [7]:
import pandas as pd
fhvhv_covid_df = pd.read_csv('/Volumes/E/data/preprocessed_data/fhvhv_covid.csv')
fhvhv_covid_df.head()

,pickup_date,num_of_SR,trip_count,avg_trip_duration,CASE_COUNT,PROBABLE_CASE_COUNT,CASE_COUNT_7DAY_AVG,ALL_CASE_COUNT_7DAY_AVG,BX_CASE_COUNT,BX_PROBABLE_CASE_COUNT,...,MN_ALL_CASE_COUNT_7DAY_AVG,QN_CASE_COUNT,QN_PROBABLE_CASE_COUNT,QN_CASE_COUNT_7DAY_AVG,QN_ALL_CASE_COUNT_7DAY_AVG,SI_CASE_COUNT,SI_PROBABLE_CASE_COUNT,SI_CASE_COUNT_7DAY_AVG,SI_ALL_CASE_COUNT_7DAY_AVG,INCOMPLETE
0,2020-09-01,34,344879,17.833136,294,34,255,278,52,9,...,48,73,4,58,62,24,1,18,19,0
1,2020-09-02,42,357969,17.718040,271,25,249,272,42,6,...,48,69,3,58,62,17,0,18,19,0
2,2020-09-03,26,391021,18.258425,274,29,250,274,41,5,...,46,90,11,62,67,17,4,19,20,0
3,2020-09-04,6,425570,19.115868,230,19,243,267,44,1,...,43,59,4,63,68,26,2,20,21,0
4,2020-09-05,0,434656,17.152276,156,11,247,269,30,3,...,43,34,3,63,68,24,0,22,23,0


In [94]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error,mean_absolute_error 
from sklearn.metrics import r2_score

from numpy import absolute
from numpy import mean
from numpy import std
from statsmodels.formula.api import *

import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [45]:
# Select a sample of interest and see if all the predictors are significant

fit = ols(
    formula="trip_count ~ num_of_SR + CASE_COUNT + PROBABLE_CASE_COUNT + CASE_COUNT_7DAY_AVG + ALL_CASE_COUNT_7DAY_AVG",
    data=fhvhv_covid_df).fit()
print(fit.summary())

                            OLS Regression Results                            
Dep. Variable:             trip_count   R-squared:                       0.291
Model:                            OLS   Adj. R-squared:                  0.260
Method:                 Least Squares   F-statistic:                     9.500
Date:                Wed, 11 Aug 2021   Prob (F-statistic):           1.29e-07
Time:                        22:29:25   Log-Likelihood:                -1491.9
No. Observations:                 122   AIC:                             2996.
Df Residuals:                     116   BIC:                             3013.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                4

# Linear regression model

In [101]:
X = fhvhv_covid_df.drop(['pickup_date','trip_count'],axis=1).values
y = fhvhv_covid_df['trip_count'].values
y = y.reshape(-1, 1)

# Standardize data
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)
scaler.fit(y)
y = scaler.transform(y)
X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=0.4, random_state=1)

lr = LinearRegression(normalize=False)
lr.fit(X_train, y_train)
pred_train_lr= lr.predict(X_train)
# calculate MAE, MSE, RMSE, R2
print("MAE on train data: ", mean_absolute_error(y_train, pred_train_lr))
print("MSE on train data: ", mean_squared_error(y_train, pred_train_lr))
print("RMSE on train data: ", np.sqrt(mean_squared_error(y_train,pred_train_lr)))
print("R2 score on train data: ", r2_score(y_train, pred_train_lr))

lr.fit(X_test, y_test)
pred_test_lr= lr.predict(X_test)
print("\nMAE on test data: ", mean_absolute_error(y_test, pred_test_lr))
print("MSE on test data: ", mean_squared_error(y_test, pred_test_lr))
print("RMSE on test data: ",np.sqrt(mean_squared_error(y_test,pred_test_lr)))
print("R2 score on test data: ", r2_score(y_test, pred_test_lr))

MAE on train data:  0.4346459736244095
MSE on train data:  0.34525932950401067
RMSE on train data:  0.5875877206885884
R2 score on train data:  0.6466229969497974

MAE on test data:  0.3252970762637669
MSE on test data:  0.17862179620494759
RMSE on test data:  0.4226367189501494
R2 score on test data:  0.8255151341918705


# Hyperparameter tuning

In [100]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedKFold

# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# define grid
grid = {'normalize':[True, False]}
# define search
search = GridSearchCV(lr, grid, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# perform the search
results = search.fit(X, y)
# summarize
print('MAE: %.3f' % results.best_score_)
print('Config: %s' % results.best_params_)

MAE: -0.746
Config: {'normalize': False}
